In [1]:
import numpy as np
import h5py
import pandas as pd
import tensorflow as tf
#from tensorflow import keras
#from sklearn.model_selection import train_test_split
from glob import glob

import matplotlib.pyplot as plt 
import seaborn as sns 

plt.rcParams["figure.figsize"] = [16,9]
sns.set_style('whitegrid')

In [2]:
df_dr3_all = pd.read_csv("/data/praveen/results/dr3_normal.csv")

In [3]:
df_dr3_all.head()

,sobject_id,0,1,2,3,4,5,6,7,8,...,4450,4451,4452,4453,4454,4455,4456,4457,4458,label
0,131116000501002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10
1,131116000501004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10
2,131116000501005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10
3,131116000501006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10
4,131116000501007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10


In [4]:
df_dr3_all["label"].value_counts()

10    581367
Name: label, dtype: int64

In [5]:
df_data_only = df_dr3_all.drop(["sobject_id","label"],1)

/tmp/ipykernel_10908/1334331202.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_data_only = df_dr3_all.drop(["sobject_id","label"],1)


In [6]:
df_data_only.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
df_data_only_inverted = 1 - df_data_only

In [8]:
import os, sys
from keras.layers import Input, Dense, Dropout
from keras.models import Model, Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.advanced_activations import PReLU
from keras import backend as K
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model

In [9]:
activation = None  # PReLU if set to None
dropout_rate = 0  # from 0 to 1
n_l_e = 5
n_epoch = 350
decoded_layer_name = 'encoded'
n_wvl = 4459
optimizer = tf.keras.optimizers.Adam()

# compute number of nodes in every connected layer
n_l_1 = int(n_wvl * 0.75)
n_l_2 = int(n_wvl * 0.50)
n_l_3 = 0  # int(n_wvl * 0.40)
n_l_4 = 0  # int(n_wvl * 0.20)
n_l_5 = int(n_wvl * 0.25)
n_l_6 = int(n_wvl * 0.10)

In [10]:
autoencoder = Sequential()

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, input_shape=(n_wvl,), activation=activation, name='E_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_1'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_1'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='E_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_2'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_2'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='E_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_3'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_3'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='E_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_4'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_4'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='E_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_5'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_5'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='E_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_6'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_6'))

autoencoder.add(Dense(n_l_e, activation=activation, name=decoded_layer_name))
if activation is None:
        autoencoder.add(PReLU(name='PR_7'))

if n_l_6 > 0:
    autoencoder.add(Dense(n_l_6, activation=activation, name='D_1'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_8'))
    if activation is None:
            autoencoder.add(PReLU(name='PR_8'))

if n_l_5 > 0:
    autoencoder.add(Dense(n_l_5, activation=activation, name='D_2'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_9'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_9'))

if n_l_4 > 0:
    autoencoder.add(Dense(n_l_4, activation=activation, name='D_3'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_10'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_10'))

if n_l_3 > 0:
    autoencoder.add(Dense(n_l_3, activation=activation, name='D_4'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_11'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_11'))

if n_l_2 > 0:
    autoencoder.add(Dense(n_l_2, activation=activation, name='D_5'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_12'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_12'))

if n_l_1 > 0:
    autoencoder.add(Dense(n_l_1, activation=activation, name='D_6'))
    if dropout_rate > 0:
        autoencoder.add(Dropout(dropout_rate, name='DO_13'))
    if activation is None:
        autoencoder.add(PReLU(name='PR_13'))

autoencoder.add(Dense(n_wvl, activation='linear', name='recreated'))
autoencoder.summary()

# Visualize network architecture and save the visualization as a file
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.pdf')
#plot_model(autoencoder, show_layer_names=True, show_shapes=True, to_file='ann_network_structure_a.png', dpi=300)

# model file handling
out_model_file = 'model_weights.h5'

if os.path.isfile(out_model_file):
    autoencoder.load_weights(out_model_file, by_name=True)


2022-04-18 10:09:09.640660: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 E_1 (Dense)                 (None, 3344)              14914240  
                                                                 
 PR_1 (PReLU)                (None, 3344)              3344      
                                                                 
 E_2 (Dense)                 (None, 2229)              7456005   
                                                                 
 PR_2 (PReLU)                (None, 2229)              2229      
                                                                 
 E_5 (Dense)                 (None, 1114)              2484220   
                                                                 
 PR_5 (PReLU)                (None, 1114)              1114      
                                                                 
 E_6 (Dense)                 (None, 445)               4

In [11]:
autoencoder.compile(optimizer=optimizer, loss='mae')

In [15]:
checkpoint = ModelCheckpoint('/data/praveen/autoencoder-run-2/'+'ann_model_run_{epoch:03d}-{loss:.4f}-{val_loss:.4f}.h5',
                                     monitor='val_loss', verbose=0, save_best_only=False,
                                     save_weights_only=True, mode='auto', period=1)

In [16]:
ann_fit_hist = autoencoder.fit(df_data_only_inverted, df_data_only_inverted,
                                       epochs=n_epoch,
                                       callbacks=[checkpoint],
                                       shuffle=True,
                                       batch_size=40000,
                                       validation_split=0.10,
                                       verbose=2)

2022-04-18 10:58:41.179820: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18664660560 exceeds 10% of free system memory.
2022-04-18 11:02:18.765669: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 18664660560 exceeds 10% of free system memory.


Epoch 1/350


2022-04-18 11:02:52.069524: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1426880000 exceeds 10% of free system memory.
2022-04-18 11:02:52.069608: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1426880000 exceeds 10% of free system memory.


14/14 - 341s - loss: 0.0352 - val_loss: 0.0289 - 341s/epoch - 24s/step
Epoch 2/350
14/14 - 275s - loss: 0.0276 - val_loss: 0.0265 - 275s/epoch - 20s/step
Epoch 3/350
14/14 - 290s - loss: 0.0259 - val_loss: 0.0255 - 290s/epoch - 21s/step
Epoch 4/350
14/14 - 289s - loss: 0.0247 - val_loss: 0.0231 - 289s/epoch - 21s/step
Epoch 5/350
14/14 - 286s - loss: 0.0213 - val_loss: 0.0196 - 286s/epoch - 20s/step
Epoch 6/350
14/14 - 289s - loss: 0.0191 - val_loss: 0.0185 - 289s/epoch - 21s/step
Epoch 7/350
14/14 - 291s - loss: 0.0183 - val_loss: 0.0182 - 291s/epoch - 21s/step
Epoch 8/350
14/14 - 286s - loss: 0.0180 - val_loss: 0.0180 - 286s/epoch - 20s/step
Epoch 9/350
14/14 - 289s - loss: 0.0179 - val_loss: 0.0179 - 289s/epoch - 21s/step
Epoch 10/350
14/14 - 292s - loss: 0.0178 - val_loss: 0.0179 - 292s/epoch - 21s/step
Epoch 11/350
14/14 - 283s - loss: 0.0178 - val_loss: 0.0178 - 283s/epoch - 20s/step
Epoch 12/350
14/14 - 288s - loss: 0.0177 - val_loss: 0.0178 - 288s/epoch - 21s/step
Epoch 13/350


In [17]:
loss_combined = np.vstack((ann_fit_hist.history['loss'], ann_fit_hist.history['val_loss'])).T
np.savetxt('ann_network_loss.txt', loss_combined)
i_best = np.argmin(ann_fit_hist.history['val_loss'])
plt.plot(ann_fit_hist.history['loss'], label='Train')
plt.plot(ann_fit_hist.history['val_loss'], label='Validation')
plt.axvline(np.arange(n_epoch)[i_best], ls='--', color='black', alpha=0.5)
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Loss value')
plt.ylim(np.nanmin(loss_combined)*0.95, np.nanpercentile(loss_combined, 99))
plt.xlim(-1, n_epoch)
plt.grid(ls='--', alpha=0.2, color='black')
plt.tight_layout()
plt.legend()
plt.savefig('ann_network_loss.png', dpi=250)
plt.close()

In [20]:
out_model_file = 'model_weights.h5'

In [21]:
print('')
        # recover weights of the selected model and compute predictions
for i_best in [10, 25, 50, 100, 150, 200, 250, 300]:

    h5_weight_files = glob('/data/praveen/autoencoder-run-2/'+'ann_model_run_{:03.0f}-*-*.h5'.format(i_best))

    if len(h5_weight_files) == 1:
        print('----------------------------------------------------------------')
        print('Restoring epoch {:.0f} with the loss ({:.4f}).'.format(i_best, ann_fit_hist.history['val_loss'][i_best-1]))
        autoencoder.load_weights(h5_weight_files[0], by_name=True)

        sub_dir = 'epoch_{:03.0f}'.format(i_best)
        os.system('mkdir ' + sub_dir)
        os.chdir(sub_dir)

        print('Saving selected model weights')
        autoencoder.save_weights(out_model_file)

        #print('Predicting values')
        #processed_data_all = autoencoder.predict(df_data_only_inverted, verbose=2, batch_size=20)

        #plot_sample_spectra()

        #print('')
        #os.chdir('..')



----------------------------------------------------------------
Restoring epoch 10 with the loss (0.0179).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 25 with the loss (0.0177).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 50 with the loss (0.0160).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 100 with the loss (0.0149).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 150 with the loss (0.0150).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 200 with the loss (0.0144).
Saving selected model weights
----------------------------------------------------------------
Restoring epoch 250 with the loss (0.0144).
Saving selected model weights
-----------------------------

In [22]:
df_dr3 = pd.read_csv("/data/praveen/results/dr3_full.csv")

In [23]:
df_dr3.head()

,sobject_id,0,1,2,3,4,5,6,7,8,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,131116000501002,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,131116000501004,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,131116000501005,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,131116000501006,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,131116000501007,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [24]:
df_dr3_data_only = df_dr3.drop(["sobject_id"],1)
df_dr3_data_only.head()
df_dr3_data_only_inverted = 1 - df_dr3_data_only

/tmp/ipykernel_10908/1450910931.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_dr3_data_only = df_dr3.drop(["sobject_id"],1)


In [25]:
df_dr3_data_only_inverted.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
print('Predicting values')
processed_data_all = autoencoder.predict(df_dr3_data_only_inverted, verbose=2, batch_size=40000)

Predicting values
15/15 - 105s - 105s/epoch - 7s/step


In [27]:
processed_data_all

array([[ 0.0007062 , -0.00076971, -0.00035594, ..., -0.00080945,
         0.00077041, -0.00093736],
       [ 0.00071003, -0.00021888, -0.00074688, ..., -0.00040598,
         0.00012151,  0.00020568],
       [ 0.00097113, -0.0002512 , -0.00065225, ..., -0.00077877,
         0.00042948, -0.00024286],
       ...,
       [ 0.00079654, -0.00054027, -0.00039906, ..., -0.00080656,
         0.00067842, -0.00044822],
       [ 0.000704  , -0.00025581, -0.00071063, ..., -0.00050182,
         0.0006199 , -0.00086663],
       [ 0.00090816, -0.00053902, -0.00033442, ..., -0.00063907,
         0.00063569, -0.00062414]], dtype=float32)

In [28]:
df_predicted_all = pd.DataFrame(processed_data_all)

In [30]:
df_predicted_all.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
0,0.000706,-0.000770,-0.000356,0.000360,-0.000050,-0.000876,0.000122,0.000082,-0.000170,-0.000392,...,0.000756,0.000128,0.000213,0.000091,0.000872,0.000331,0.000736,-0.000809,0.000770,-0.000937
1,0.000710,-0.000219,-0.000747,0.000530,0.000823,-0.000009,0.000586,0.000549,-0.000094,-0.000656,...,0.000082,-0.000362,-0.000344,-0.000076,0.000664,-0.000703,0.000716,-0.000406,0.000122,0.000206
2,0.000971,-0.000251,-0.000652,0.000780,0.000541,-0.000482,0.000262,0.000376,0.000391,-0.000594,...,0.000400,-0.000088,0.000155,0.000066,0.000700,-0.000617,0.000331,-0.000779,0.000429,-0.000243
3,0.001088,-0.000359,-0.000646,0.000193,0.000507,-0.000113,0.000173,0.000856,0.000919,-0.000589,...,0.000086,-0.000373,-0.000424,0.000150,0.000460,-0.000289,0.000019,-0.000651,0.000062,-0.000111
4,0.000690,-0.000478,-0.000466,0.000209,0.000357,-0.000235,0.000467,0.000570,0.000634,-0.000537,...,0.000138,-0.000177,-0.000386,0.000305,0.000235,-0.000514,0.000277,-0.000678,0.000737,-0.000651


In [31]:
df_predicted_all.to_csv("/data/praveen/results/dr3_full_predicted_inverted.csv",index=None)